In [1]:
#Loading in data and importing necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('drive')
#!wget -q --show-progress "https://zenodo.org/record/4355693/files/USFD_v1.0.csv"

Mounted at drive


In [2]:
# Load in dataset as a pandas dataframe
flood_data_raw = pd.read_csv("drive/My Drive/USFD_v1.0.csv")

<ipython-input-2-ed327fc74404>:2: DtypeWarning: Columns (1,2,8,13,14,16,17,25) have mixed types. Specify dtype option on import or set low_memory=False.
  flood_data_raw = pd.read_csv("drive/My Drive/USFD_v1.0.csv")


In [3]:
print("Dimensions: " + str(flood_data_raw.shape))
np.sum(flood_data_raw.isnull())

Dimensions: (698507, 26)


ID                    0
DATE_BEGIN            1
DATE_END           5155
DURATION           4975
LON               26083
LAT               26084
COUNTRY               0
STATE               397
LOCATION         553928
AREA             603600
FATALITY         684429
DAMAGE           554045
SEVERITY         697812
SOURCE           567266
CAUSE            608815
SOURCE_DB             0
SOURCE_ID          4927
DESCRIPTION      571959
slope             26661
dem               27051
LULC              26185
DISTANT_RIVER     26131
CONT_AREA        172776
DEPTH            632051
year             541964
geometry         156415
dtype: int64

In [4]:
flood_data_drop = flood_data_raw.dropna(subset=["DAMAGE", "STATE", "LAT","LON", "year"])
flood_data_zero = flood_data_drop.drop(flood_data_drop[flood_data_drop.DAMAGE < 1e3].index)
flood_data_zero = flood_data_zero.drop(flood_data_zero[flood_data_zero.DAMAGE >= 1e10].index)
print("Dimensions: ", flood_data_zero.shape)
np.sum(flood_data_zero.isnull())

Dimensions:  (41701, 26)


ID                   0
DATE_BEGIN           0
DATE_END             0
DURATION             5
LON                  0
LAT                  0
COUNTRY              0
STATE                0
LOCATION             4
AREA              9662
FATALITY         38114
DAMAGE               0
SEVERITY         41701
SOURCE            2525
CAUSE            10222
SOURCE_DB            0
SOURCE_ID            0
DESCRIPTION        768
slope              133
dem                252
LULC                16
DISTANT_RIVER        1
CONT_AREA         3969
DEPTH            37569
year                 0
geometry         41655
dtype: int64

In [5]:
flood_data = flood_data_zero.drop(["ID", "DATE_BEGIN", "DATE_END", "DESCRIPTION", "SOURCE_DB", "SOURCE_ID", "COUNTRY", "LOCATION", "geometry", "SEVERITY", "SOURCE"], axis = 1)
print("Dimensions: ", flood_data.shape)
np.sum(flood_data.isnull())
flood_data.head()

Dimensions:  (41701, 15)


,DURATION,LON,LAT,STATE,AREA,FATALITY,DAMAGE,CAUSE,slope,dem,LULC,DISTANT_RIVER,CONT_AREA,DEPTH,year
19,0.0,-73.668698,44.727894,NEW YORK,NaN,NaN,100000.0,NaN,1.236463,366.0,114.0,3.255651,0.260616,NaN,1996.0
28,7.0,-84.512460,39.101454,OHIO,NaN,NaN,1000000.0,NaN,6.530572,191.0,50.0,0.867011,0.154607,NaN,1996.0
34,1.0,-82.570251,38.606546,OHIO,NaN,NaN,700000.0,NaN,13.005761,232.0,114.0,5.422984,0.357606,NaN,1996.0
35,1.0,-82.570251,38.606546,OHIO,NaN,NaN,250000.0,NaN,13.005761,232.0,114.0,5.422984,0.357606,NaN,1996.0
36,1.0,-82.570251,38.606546,OHIO,NaN,NaN,250000.0,NaN,13.005761,232.0,114.0,5.422984,0.357606,NaN,1996.0


In [6]:
# Should I impute if damage is 0
flood_data['DURATION'].fillna(flood_data['DURATION'].mean(), inplace=True)
flood_data['AREA'].fillna(flood_data['AREA'].mean(), inplace=True)
flood_data['FATALITY'].fillna(flood_data['FATALITY'].mean(), inplace=True)
flood_data['slope'].fillna(flood_data['slope'].mean(), inplace=True)
flood_data['dem'].fillna(flood_data['dem'].mean(), inplace=True)
flood_data['LULC'].fillna(flood_data['LULC'].mean(), inplace=True)
flood_data['DISTANT_RIVER'].fillna(flood_data['DISTANT_RIVER'].mean(), inplace=True)
flood_data['CONT_AREA'].fillna(flood_data['CONT_AREA'].mean(), inplace=True)
flood_data['DEPTH'].fillna(flood_data['DEPTH'].mean(), inplace=True)
flood_data['CAUSE'].fillna("Unknown", inplace=True)
np.sum(flood_data.isnull())
np.sum(flood_data["DAMAGE"] == 0)
len(flood_data["STATE"])
print(flood_data["STATE"])

19        NEW YORK
28            OHIO
34            OHIO
35            OHIO
36            OHIO
            ...   
158768    MICHIGAN
158769    MICHIGAN
158800    MICHIGAN
158823    MICHIGAN
158825    MICHIGAN
Name: STATE, Length: 41701, dtype: object


In [7]:
#Numerical Transformation
state_dummies = pd.get_dummies(flood_data.STATE)
flood_data_states = pd.concat([flood_data, state_dummies], axis=1)
cause_dummies = pd.get_dummies(flood_data_states.CAUSE)
flood_data_causes = pd.concat([flood_data_states, cause_dummies], axis=1)
flood_data_preproc = flood_data_causes.drop(["STATE", "CAUSE"], axis = 1)

In [8]:
print("Dimensions: " + str(flood_data_preproc.shape))
np.sum(flood_data_preproc.isnull())

Dimensions: (41701, 75)


DURATION                 0
LON                      0
LAT                      0
AREA                     0
FATALITY                 0
                        ..
dam break,dam release    0
heavy rain               0
heavy rain,snowmelt      0
ice jam                  0
snowmelt                 0
Length: 75, dtype: int64

In [9]:
# cause_dummies = pd.get_dummies(flood_data.CAUSE)
# flood_data_causes = pd.concat([flood_data, cause_dummies], axis=1)
# flood_data_preproc = flood_data_causes.drop(["CAUSE"], axis = 1)

In [10]:
# saving the dataframe
flood_data_preproc.to_csv("floodPreproc.csv", index=False)
!cp floodPreproc.csv "drive/My Drive/"